In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import re

from scipy.spatial import cKDTree

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [2]:
trees2012 = pd.read_csv("2012_Mathced_NeighbourStats.csv")
trees2021 = pd.read_csv("2021_Matched.csv")
trees2021.rename(columns={'Unnamed: 0': 'MatchingID'}, inplace=True)
trees2012

,Unnamed: 0,TreeID,TreePosX,TreePosY,TreePosZ,TreeHeight(metre),DBH(metre),CrownDiameter(metre),CrownDiameter(S-N)(metre),CrownDiameter(E-W)(metre),CrownArea(square metre),CrownVolume(cubic metre),OldID,FutureIndex,No of Neighbouring Trees,Neighbour Mean Height,Neighbour max Height,Neighbour min Height,Neighbour Mean Location,Neighbour Mean Angle
0,391814,391815,246301.44,701614.75,35.710,35.7,0.0,8.2,11.5,8.3,52.6,67.5,392154,848202,8,33.576250,36.800,32.110,3.966789,89.550631
1,391906,391907,246304.34,701616.32,36.750,36.8,0.0,7.5,9.4,6.6,44.7,56.9,392054,848166,11,33.157273,37.200,32.110,3.841467,73.723389
2,392006,392007,246318.34,701613.54,34.010,34.0,0.0,8.2,11.4,7.5,53.5,78.6,392187,848212,7,33.905714,37.200,32.150,4.694842,128.362003
3,392095,392096,246398.71,701610.47,38.330,38.3,0.0,6.9,8.6,8.2,37.1,43.9,392412,848604,6,35.446667,39.710,32.130,4.387008,136.702138
4,392135,392136,246391.82,701610.63,39.710,39.7,0.0,6.1,5.0,9.0,29.3,35.8,392460,848479,8,34.490000,39.320,32.130,4.613993,60.020659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490150,3941595,737200,245306.44,700024.91,3.364,3.4,0.0,1.2,1.3,1.8,1.1,0.5,2383418,1001304,4,3.605750,3.904,3.386,3.787542,85.153445
490151,3941596,737210,245287.02,700024.34,3.528,3.5,0.0,3.4,3.0,3.9,9.1,13.3,2383446,960386,8,3.804000,5.036,3.225,2.389412,234.329141
490152,3941600,737250,245187.96,700013.57,18.948,18.9,0.0,8.2,11.9,8.0,52.7,130.3,2383802,960701,3,9.092667,19.054,3.088,3.644109,76.106757
490153,3941602,737270,245070.60,700012.06,3.345,3.3,0.0,1.1,1.4,1.0,0.9,0.9,2383830,960882,5,3.967600,4.546,3.084,3.871242,193.240520


In [3]:
def createcKD(df):
    xs = df["X"]
    ys = df["Y"]

    cords = list(zip(xs,ys))

    return cKDTree(cords)

cells = pd.read_excel("D:\DissertationData\Data2012_Joined_JuanSuarez20250211_ver2.xlsx")
cellsCKD = createcKD(cells)
cells

<>:9: SyntaxWarning: invalid escape sequence '\D'
<>:9: SyntaxWarning: invalid escape sequence '\D'
C:\Users\stanc\AppData\Local\Temp\ipykernel_15368\1214044832.py:9: SyntaxWarning: invalid escape sequence '\D'
  cells = pd.read_excel("D:\DissertationData\Data2012_Joined_JuanSuarez20250211_ver2.xlsx")


,Join_Count,TARGET_FID,X,Y,XSize,YSize,CanopyCove,GapFractio,LAI,Top_Height,...,comp_maxma,shape_Le_1,shape_Area,GlobalID,OBJECTID_2,rel_fw_gui,FREQUENCY,MAX_areap,FIRST_spac,FIRST_spis
0,1,186749,250885.000,700165,30,30,0.892,0.108118,4.449,18.500000,...,0,1719.574480,97850.010623,{94733332-7252-4759-B87B-9A813B452A56},509,{1B82B651-5EBD-4A60-A2D7-16595A21028A},1,100,1.8,SS
1,1,9411,255268.844,697725,30,30,0.919,0.081010,5.026,18.500000,...,0,1478.899684,37215.048844,{16D0CF81-59FF-411A-AAA8-659953A8A67F},5,{007D445F-84CA-4CC1-9B5F-BE86242D5183},4,40,2.0,SS
2,1,140742,249505.000,700705,30,30,0.509,0.491063,1.422,49.786722,...,0,4413.995565,99344.767986,{2BC790AF-BEA5-44C9-B694-6DEE83D7C144},801,{2C5EEA1E-F90E-4A35-86C4-F43CE277FF8B},9,21,1.7,SS
3,1,141000,249475.000,700675,30,30,0.632,0.367886,2.000,48.622770,...,0,4413.995565,99344.767986,{2BC790AF-BEA5-44C9-B694-6DEE83D7C144},801,{2C5EEA1E-F90E-4A35-86C4-F43CE277FF8B},9,21,1.7,SS
4,1,148546,248335.000,700345,30,30,0.947,0.052632,5.889,44.092846,...,0,1100.147755,49031.293305,{0EAFCBC3-0415-45E0-8EDD-3BE7A8D38EDD},2527,{869F50EC-3920-42AA-9988-021775E38FBA},4,42,2.0,SS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32523,1,193210,250015.000,702235,30,30,1.000,0.000364,15.835,24.272940,...,0,4008.799720,275528.056710,{2C57FC65-F904-42B7-8103-149A32C35766},4162,{DEDED01D-4AE0-483C-8447-AC695798D006},1,100,2.0,SS
32524,1,193211,250015.000,702205,30,30,0.989,0.010950,9.029,23.649176,...,0,4008.799720,275528.056710,{2C57FC65-F904-42B7-8103-149A32C35766},4162,{DEDED01D-4AE0-483C-8447-AC695798D006},1,100,2.0,SS
32525,1,193212,250015.000,702175,30,30,0.842,0.157753,3.693,23.648153,...,0,1923.187649,31796.844368,{EE052C5D-2C8C-42A6-805B-7CCC6C24F461},1639,{56CBB887-8EA7-443B-BBBE-426BC2A9978F},1,100,2.0,SS
32526,1,193216,250015.000,702055,30,30,0.540,0.459860,1.554,20.004869,...,0,1923.187649,31796.844368,{EE052C5D-2C8C-42A6-805B-7CCC6C24F461},1639,{56CBB887-8EA7-443B-BBBE-426BC2A9978F},1,100,2.0,SS


In [4]:
filtered2012 = trees2012[["TreePosX", "TreePosY", "TreePosZ", "CrownDiameter(metre)", "CrownVolume(cubic metre)", "No of Neighbouring Trees", "Neighbour Mean Height", "Neighbour max Height", "Neighbour min Height", "Neighbour Mean Location","Neighbour Mean Angle","FutureIndex"]]

canopyCov = []
lai = []
topHeight = []
age = []
yc = []
meanDBH = []
basalArea = []
volHa = []


for index, row in filtered2012.iterrows():
    _, cellIndex = cellsCKD.query((row["TreePosX"], row["TreePosY"]))

    cell = cells.iloc[cellIndex]

    canopyCov.append(cell["CanopyCove"])
    lai.append(cell["LAI"])
    topHeight.append(cell["Top_Height"])
    age.append(cell["Age"])
    yc.append(cell["YieldClass"])
    meanDBH.append(cell["MeanDBH_cm"])
    basalArea.append(cell["Basal_Area_m2"])
    volHa.append(cell["Vol_ha_m3"])

filtered2012["Canopy Coverage"] = canopyCov
filtered2012["LAI"] = lai
filtered2012["Top Height"] = topHeight
filtered2012["Age"] = age
filtered2012["YC"] = yc
filtered2012["meanDBH"] = meanDBH
filtered2012["Basal Area"] = basalArea
filtered2012["Vol Ha"] = volHa    

C:\Users\stanc\AppData\Local\Temp\ipykernel_15368\1653865124.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered2012["Canopy Coverage"] = canopyCov
C:\Users\stanc\AppData\Local\Temp\ipykernel_15368\1653865124.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered2012["LAI"] = lai
C:\Users\stanc\AppData\Local\Temp\ipykernel_15368\1653865124.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [5]:
#Removing unneeded columns

#Min max normalizing to range -1 to 1
filtered2012["Canopy CoverageNorm"] = (((filtered2012["Canopy Coverage"] - filtered2012["Canopy Coverage"].min()) / (filtered2012["Canopy Coverage"].max() - filtered2012["Canopy Coverage"].min())) -0.5) * 2
filtered2012["TreePosXNorm"] = (((filtered2012["TreePosX"] - filtered2012["TreePosX"].min()) / (filtered2012["TreePosX"].max() - filtered2012["TreePosX"].min())) -0.5) * 2
filtered2012["TreePosYNorm"] = (((filtered2012["TreePosY"] - filtered2012["TreePosY"].min()) / (filtered2012["TreePosY"].max() - filtered2012["TreePosY"].min())) -0.5) * 2
filtered2012["Neighbour Mean AngleNorm"] = ((filtered2012["Neighbour Mean Angle"]/360) -0.5) * 2

scaler = StandardScaler()

filtered2012[["TreePosZNorm",
               "CrownDiameter(metre)Norm", 
               "CrownVolume(cubic metre)Norm", 
               "No of Neighbouring TreesNorm", 
               "Neighbour Mean HeightNorm", 
               "Neighbour max HeightNorm", 
               "Neighbour min HeightNorm", 
               "Neighbour Mean LocationNorm",
               "LAINorm",
               "Top HeightNorm",
               "AgeNorm",
               "YCNorm",
               "meanDBHNorm",
               "Basal AreaNorm",
               "Vol HaNorm"]] = scaler.fit_transform(filtered2012[["TreePosZ",
                                                                    "CrownDiameter(metre)", 
                                                                    "CrownVolume(cubic metre)", 
                                                                    "No of Neighbouring Trees", 
                                                                    "Neighbour Mean Height", 
                                                                    "Neighbour max Height", 
                                                                    "Neighbour min Height", 
                                                                    "Neighbour Mean Location",
                                                                    "LAI",
                                                                    "Top Height",
                                                                    "Age",
                                                                    "YC",
                                                                    "meanDBH",
                                                                    "Basal Area",
                                                                    "Vol Ha"]])

filtered2012

,TreePosX,TreePosY,TreePosZ,CrownDiameter(metre),CrownVolume(cubic metre),No of Neighbouring Trees,Neighbour Mean Height,Neighbour max Height,Neighbour min Height,Neighbour Mean Location,...,Neighbour max HeightNorm,Neighbour min HeightNorm,Neighbour Mean LocationNorm,LAINorm,Top HeightNorm,AgeNorm,YCNorm,meanDBHNorm,Basal AreaNorm,Vol HaNorm
0,246301.44,701614.75,35.710,8.2,67.5,8,33.576250,36.800,32.110,3.966789,...,2.463972,2.974115,2.032540,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
1,246304.34,701616.32,36.750,7.5,56.9,11,33.157273,37.200,32.110,3.841467,...,2.512235,2.974115,1.918680,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
2,246318.34,701613.54,34.010,8.2,78.6,7,33.905714,37.200,32.150,4.694842,...,2.512235,2.979707,2.694007,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
3,246398.71,701610.47,38.330,6.9,43.9,6,35.446667,39.710,32.130,4.387008,...,2.815087,2.976911,2.414327,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
4,246391.82,701610.63,39.710,6.1,35.8,8,34.490000,39.320,32.130,4.613993,...,2.768031,2.976911,2.620552,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490150,245306.44,700024.91,3.364,1.2,0.5,4,3.605750,3.904,3.386,3.787542,...,-1.505198,-1.041446,1.869687,-1.003287,1.932923,1.274806,0.614708,1.903775,1.770711,2.026110
490151,245287.02,700024.34,3.528,3.4,13.3,8,3.804000,5.036,3.225,2.389412,...,-1.368613,-1.063954,0.599427,-1.003287,1.932923,1.274806,0.614708,1.903775,1.770711,2.026110
490152,245187.96,700013.57,18.948,8.2,130.3,3,9.092667,19.054,3.088,3.644109,...,0.322773,-1.083106,1.739372,-0.872018,-0.066280,1.274806,-1.085545,-0.157285,0.023850,-0.181586
490153,245070.60,700012.06,3.345,1.1,0.9,5,3.967600,4.546,3.084,3.871242,...,-1.427736,-1.083666,1.945732,-0.872018,-0.066280,1.274806,-1.085545,-0.157285,0.023850,-0.181586


In [6]:


cleaned2021 = trees2021.drop_duplicates(subset=["MatchingID"])

matchedShuffle2021 = filtered2012[['FutureIndex']].merge(
    cleaned2021, left_on='FutureIndex', right_on='MatchingID', how='left'
)
matchedShuffle2021


matchedYears = matchedShuffle2021[["TreePosZ", "CrownDiameter(metre)","CrownArea(square metre)","CrownVolume(cubic metre)", "MatchingID"]].merge(
    filtered2012, left_index=True, right_index=True
)

matchedYears

,TreePosZ_x,CrownDiameter(metre)_x,CrownArea(square metre),CrownVolume(cubic metre)_x,MatchingID,TreePosX,TreePosY,TreePosZ_y,CrownDiameter(metre)_y,CrownVolume(cubic metre)_y,...,Neighbour max HeightNorm,Neighbour min HeightNorm,Neighbour Mean LocationNorm,LAINorm,Top HeightNorm,AgeNorm,YCNorm,meanDBHNorm,Basal AreaNorm,Vol HaNorm
0,NaN,NaN,NaN,NaN,NaN,246301.44,701614.75,35.710,8.2,67.5,...,2.463972,2.974115,2.032540,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
1,5.705,6.2,29.8,92.9,848166.0,246304.34,701616.32,36.750,7.5,56.9,...,2.512235,2.974115,1.918680,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
2,NaN,NaN,NaN,NaN,NaN,246318.34,701613.54,34.010,8.2,78.6,...,2.512235,2.979707,2.694007,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
3,NaN,NaN,NaN,NaN,NaN,246398.71,701610.47,38.330,6.9,43.9,...,2.815087,2.976911,2.414327,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
4,NaN,NaN,NaN,NaN,NaN,246391.82,701610.63,39.710,6.1,35.8,...,2.768031,2.976911,2.620552,-0.932395,-0.487961,1.737995,-1.549242,-0.578980,-0.421176,-0.292837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490150,NaN,NaN,NaN,NaN,NaN,245306.44,700024.91,3.364,1.2,0.5,...,-1.505198,-1.041446,1.869687,-1.003287,1.932923,1.274806,0.614708,1.903775,1.770711,2.026110
490151,NaN,NaN,NaN,NaN,NaN,245287.02,700024.34,3.528,3.4,13.3,...,-1.368613,-1.063954,0.599427,-1.003287,1.932923,1.274806,0.614708,1.903775,1.770711,2.026110
490152,NaN,NaN,NaN,NaN,NaN,245187.96,700013.57,18.948,8.2,130.3,...,0.322773,-1.083106,1.739372,-0.872018,-0.066280,1.274806,-1.085545,-0.157285,0.023850,-0.181586
490153,NaN,NaN,NaN,NaN,NaN,245070.60,700012.06,3.345,1.1,0.9,...,-1.427736,-1.083666,1.945732,-0.872018,-0.066280,1.274806,-1.085545,-0.157285,0.023850,-0.181586


In [7]:
matchedYears.to_csv("D:\DissertationData\matchedTreesInputsStandardized.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\stanc\AppData\Local\Temp\ipykernel_15368\675579740.py:1: SyntaxWarning: invalid escape sequence '\D'
  matchedYears.to_csv("D:\DissertationData\matchedTreesInputsStandardized.csv")


In [8]:
total = len(matchedYears)

split = math.floor(total * 0.8)

shuffled = matchedYears.sample(frac=1)

train = shuffled.iloc[:split]

test = shuffled.iloc[split:]


In [9]:
train_x = train[["TreePosX", "TreePosY", "TreePosZ_y", "CrownDiameter(metre)_y", "CrownVolume(cubic metre)_y", "No of Neighbouring Trees","Neighbour Mean Height","Neighbour max Height","Neighbour min Height","Neighbour Mean Location","Neighbour Mean Angle"]]
train_y = train[["TreePosZ_x","CrownDiameter(metre)_x", "CrownArea(square metre)", "CrownVolume(cubic metre)_x"]]

test_x = test[["TreePosX", "TreePosY", "TreePosZ_y", "CrownDiameter(metre)_y", "CrownVolume(cubic metre)_y", "No of Neighbouring Trees","Neighbour Mean Height","Neighbour max Height","Neighbour min Height","Neighbour Mean Location","Neighbour Mean Angle"]]
test_y = test[["TreePosZ_x","CrownDiameter(metre)_x", "CrownArea(square metre)", "CrownVolume(cubic metre)_x"]]

train_x = train_x.to_numpy()
train_y = train_y.to_numpy()

test_x = test_x.to_numpy()
test_y = test_y.to_numpy()

test_y[0].shape


(4,)

In [10]:
model = Sequential([
    Dense(256, activation="relu", input_shape=(11,)),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),  
    Dense(64, activation="relu"),  
    Dense(4, activation="linear")  
])

C:\Users\stanc\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
err_test = test.sample(frac=1)

errtest_x = err_test[["TreePosX", "TreePosY", "TreePosZ_y", "CrownDiameter(metre)_y", "CrownVolume(cubic metre)_y", "No of Neighbouring Trees","Neighbour Mean Height","Neighbour max Height","Neighbour min Height","Neighbour Mean Location","Neighbour Mean Angle"]]
errtest_y = err_test[["TreePosZ_x","CrownDiameter(metre)_x", "CrownVolume(cubic metre)_x"]]

errtest_x.to_numpy()
errtest_y.to_numpy()

for errTx in errtest_x:
    pred_y = model.predict(errTx)


ValueError: Unrecognized data type: x=TreePosX (of type <class 'str'>)